In [1]:
from gurobipy import *
from math import factorial
import pandas as pd
import time
import numpy as np
from matplotlib import pyplot as plt
import random

### Parameters setting

In [2]:
coord = {}
n_seats = 188

# Defines the coordinates of each seat
for i in range(1,n_seats+1):
    if i < 33:
        coord[i] = [1,i]
    elif i < 65:
        coord[i] = [2,i-32]
    elif i < 96:
        coord[i] = [3,i-64]
    elif i < 127:
        coord[i] = [5,i-95]
    elif i < 158:
        coord[i] = [6,i-126]
    elif i < 189:
        coord[i] = [7,i-157]
        
# Calculates the Manhattan distance between seats
d = {}
for i in range(1,n_seats+1):
    for j in range(1,n_seats+1):
        d[i,j] = abs(coord[i][0] - coord[j][0]) + abs(coord[i][1]-coord[j][1])
    
    
# Imports data from Excel file and drops Nan
# (Only the data of bought seats is used)
df = pd.read_excel (r'BD SEAT .xlsx').dropna()


# Retrieves the seat most times purchased
maximo = max(df['UnitDesignator'].value_counts())

# Rank 
normalizado = df['UnitDesignator'].value_counts()/maximo


# Seats are encoded by 1A, 1B,... to 1, 2, ...
vec = {}
for i in range(0, len(normalizado.index)):
    if len(normalizado.index[i]) == 2 :
        if normalizado.index[i][-1] == 'A':
            vec[int(normalizado.index[i][0])] = 1
        elif normalizado.index[i][-1] == 'B':
            vec[int(normalizado.index[i][0])+32] = 1
        elif normalizado.index[i][-1] == 'C':
            vec[int(normalizado.index[i][0])+32+32] = 1
        elif normalizado.index[i][-1] == 'D':
            vec[int(normalizado.index[i][0])+32+32+31] = 1
        elif normalizado.index[i][-1] == 'E':
            vec[int(normalizado.index[i][0])+32+32+31+31] = 1
        elif normalizado.index[i][-1] == 'F':
            vec[int(normalizado.index[i][0])+32+32+31+31+31] = 1
            
    elif len(normalizado.index[i]) == 3 :
        if normalizado.index[i][-1] == 'A':
            vec[int(normalizado.index[i][0:2])] = 1
        elif normalizado.index[i][-1] == 'B':
            vec[int(normalizado.index[i][0:2])+32] = 1
        elif normalizado.index[i][-1] == 'C':
            vec[int(normalizado.index[i][0:2])+32+32] = 1
        elif normalizado.index[i][-1] == 'D':
            vec[int(normalizado.index[i][0:2])+32+32+31] = 1
        elif normalizado.index[i][-1] == 'E':
            vec[int(normalizado.index[i][0:2])+32+32+31+31] = 1
        elif normalizado.index[i][-1] == 'F':
            vec[int(normalizado.index[i][0:2])+32+32+31+31+31] = 1

# Sets rank to each seat            
contador = -1
for i in vec.keys():
    contador += 1
    vec[i] = normalizado.values[contador]
    
df = pd.read_excel (r'BD SEAT .xlsx')
df['BookingBookDate']= pd.to_datetime(df['BookingBookDate'])
df = df[df['SeatBookDateTime'].isnull()]
dep = df['DepartureDate']
depDates = df['DepartureDate'].unique()




# Minimum distance parameter
beta = {}

# Minimum distance
min_dist = 7 # change to 0 if minimising seat's distance
delta = min_dist

# Percentage of blocked seats
ghost_percentage = 0.8


flightCounter = 0

# Objective cost
costo_objetivo = []

# Objective distance
distancia_objetivo = []

# Balance tuneable parameter
balance_param = 30




In [3]:
I = range(1, n_seats + 1)

c = {}
    
vecc = [39,34,34,34,34,27,27,27,27,27,27,29,29,18,18,18,18,18,18,18,18,18,18,14,14,14,14,14,14,14,14,14]
vec2 = [34,29,29,29,29,22,22,22,22,22,22,24,24,12,12,12,12,12,12,12,12,12,12,9,9,9,9,9,9,9,9,9]


for i in range(0,32):
    c[i+1] = vecc[i]
    c[i+33] = vec2[i]
    if i != 31:
        c[i+65] = vecc[i]
        c[i+96] = vecc[i]
        c[i+127] = vec2[i]
        c[i+158] = vecc[i]


costoReal = c.copy()

bono = 100
for i in c:
    if i != 32:
        c[i] = c[i] + bono*vec[i]

c[32] = 500
c[64] = 500
c[31] = 500
c[63] = 500
c[95] = 500
c[126] = 500
c[157] = 500
c[188] = 500
c[31] = 500
c[62] = 500
c[94] = 500
c[125] = 500   
c[156] = 500
c[187] = 500  



for i in I:
    for j in I:
        if d[i,j] >= delta:
            beta[i,j] = 1
        else:
            beta[i,j] = 0

# Optimisation model function

In [4]:
def optimisation(q, occupancy):    
        
    m = Model("Proyecto: "+str(t))
    m.setParam("OutputFlag",0)
    
    
    w1 = 1.8
    w2 = -1.5
    
    

    
    # Decision variables
    x = {(i):m.addVar(vtype=GRB.BINARY, obj = w1*c[i], name="x"+str(i)) for i in I}
    y = {(i,j):m.addVar(vtype=GRB.BINARY, obj = w2*d[i,j], name="y"+str(i)+str(j)) for i in I for j in I if j!=i}



    # Constraints
    for i in I:
        

        # Constraints (5)
        m.addConstr(x[i] <= a[i])
        
        # Constraints (3) and (4)
        m.addConstr(quicksum(y[i,m] for m in I if m!=i) - x[i]*(q-1) == 0)
        m.addConstr(quicksum(y[m,i] for m in I if m!=i) - x[i]*(q-1) == 0)


    # Constraint (2)
    m.addConstr(quicksum(x[i] for i in I) == q)



    if q >= 10:

        m.setParam('SolutionLimit', 1)
        m.setParam('MIPGap', 0.2)
         

    elif q < 10 and q >= 7: 

        m.setParam('MIPGap', 0.15)


    elif q < 7:

        m.setParam('MIPGap', 0.05)
 

    for i in I:
            for j in I:
                if i!=j:
                    # Constraints (6)
                    m.addConstr(y[i,j] <= beta[i,j])
                    
    #if occupancy >= 40 and occupancy <= 70:
    #    m.addConstr(quicksum(assigned[i]*l[i] + x[i]*l[i] for i in I) >= -balance_param)
    #    m.addConstr(quicksum(assigned[i]*l[i] + x[i]*l[i] for i in I) <= balance_param)
    #    m.addConstr(quicksum(assigned[i]*r[i] + x[i]*r[i] for i in I) >= -balance_param)
    #    m.addConstr(quicksum(assigned[i]*r[i] + x[i]*r[i] for i in I) <= balance_param)
    
    m.ModelSense = 1 # Minimising

    
    m.setParam('Heuristics', 0)
    m.setParam('Threads', 8)
    m.setParam("DualReductions",0)
    m.setParam("InfUnbdInfo", 1)
    
    
    m.update()

    m.optimize()
    
    if m.Status == 2:
        return m.objVal, x, y, m.Status, m
    if m.Status == 3:
        return 'inf', x, y, m.Status, m

Initial seats assigned

In [5]:
a = {}
assigned = {}
for i in I:
    a[i] = 1
    
    
initial_occupation = 0#0.8 # Percentage of seats initially assigned
filled = True

if initial_occupation > 0:
    filled = False 
    
    
filled_seats = []

while filled == False:
    
    rand_n = random.randint(1, n_seats)

    if rand_n not in filled_seats:
        filled_seats.append(rand_n)
        a[rand_n] = 0
    
    if len(filled_seats) == np.floor(initial_occupation * n_seats):
        filled = True
        
        
assigned = dict(a)

In [6]:
l = {}
for i in I:
    if i <= 32:
        l[i] = -3
    elif i <= 64:
        l[i] = -2
    elif i <= 95:
        l[i] = -1
    elif i <= 126:
        l[i] = 1
    elif i <= 157:
        l[i] = 2
    elif i <= 188:
        l[i] = 3
        
rows = 32
r = {}

            
for i in range(1, rows + 1):
    for j in range(1, 7):
        if j <= 3:
            
            if i <= rows/2:
                r[i + rows * (j - 1)] = rows/2 - (i - 1) 
            else:
                r[i + rows * (j - 1)] = rows/2 - i
                
        else:
            if i <= rows/2:
                r[i + (rows - 1) * (j - 1) + 2] = rows/2 - (i - 1) 
            else:
                r[i + (rows - 1) * (j - 1) + 2] = (rows/2 - i)
r[127] = 16.0
r[158] = 16.0
r[96] = 16.0
r.pop(189)

-16.0

# Algorithm 

In [7]:

for dep_date in depDates:#[0:1]:
    
    dep_df = df[df['DepartureDate'] == dep_date]
    NumberFlights = dep_df['FlightNumber'].unique()

    
    for number_flight in NumberFlights:#[5:6]:
        
        
        
        
        
        number_df = dep_df[dep_df['FlightNumber'] == number_flight]
        
        fec = pd.DataFrame(number_df['BookingBookDate'].value_counts())
        fec['Date'] = fec.index
        qReal = fec.sort_values(by='Date', ascending=True)['BookingBookDate']
        
        flightCounter += 1
        
        
        qR = []
        for i in qReal:
            qR.append(i)
            
            
        #### Parametro para porcentaje de sillas ocupadas inicialmente
        
        sortedC = dict(sorted(c.items(), key=lambda x:x[1]))

        cntr = 0
        indexes = []
        for i in reversed(sortedC):
            if cntr <= len(c)*ghost_percentage-1:
                a[i] = 0
                indexes.append(i)
            cntr += 1
        indexes = list(reversed(indexes))           
            
 

        #### Empieza a optimizar
    
        mean_OF = []
        occupied = 0
        seats = {}
        tInicial = time.time()
        
        iter_time = {}
        t = 0
        iter_OF = {}
        counter_ = 0
        
        
        superCounter = 0
        while sum(a.values()) > 0:
            t_Iter = time.time()
            
            
            if len(qR) - t <= 0:
                break
            
            
            t += 1
            
            breakVar = 0
            while qR[t-1] >= 20:
                t += 1
                if len(qR) - t <= 0:
                    breakVar = 1
                    break
                    
            if breakVar == 1:
                break
                
            q = qR[t-1]  
            #q = 4
            
            
            occupied = initial_occupation*100 + len(seats)/n_seats * 100
            OF, x, y, m_status, m = optimisation(q, occupied)
            
            if m_status == 3 and delta > 0:
                
                t -= 1
                
                delta -= 1
                
                for i in I:
                    for j in I:
                        if d[i,j] >= delta:
                            beta[i,j] = 1
                        else:
                            beta[i,j] = 0
                            
                            
            elif m_status == 3 and delta <= 0:
                break
            else:
                iter_time[t] = round(time.time() - t_Iter, 4)
                iter_OF[t] = OF
                
                
                distancia_objetivo.append(sum(y[i,j].X*(d[i,j]) for i in I for j in I if i!=j))
                costo_objetivo.append(sum(x[i].X*c[i] for i in I))

                delta = min_dist
                
                
                
                
                vals = { k : v.X for k,v in x.items() }
                for b in vals:
                    if vals[b] > 0.5:
                        a[b] = 0
                        seats[b] = t
                        assigned[b] = 0

                if superCounter <= len(indexes):
                    for i in indexes[superCounter:superCounter + q]:
                        a[i] = 1

                    superCounter += q
                #print('Time:' , iter_time[t], 'Occupation %:', round(occupied,6), 'Blocked seats %:', 100*round(1-sum(a.values())/188, 6)) 
    
        tTotal = round(time.time() - tInicial,5) 
        
        #if len(iter_OF.values()) > 0:
            #mean_OF.append(sum(iter_OF.values())/len(iter_OF.values()))
            #'Mean OF:', mean_OF, dep_date, number_flight,
        print('Time in seconds:', tTotal, 'OF sum:', round(sum(iter_OF.values()),4), 'Occupation %:', round(occupied, 6))
        
        results = pd.DataFrame(index = list(range(1,33)),columns = ['A','B','C','D','E','F'])

        for i in seats.keys():
            if i <= 32:
                results['A'][i] = seats[i]
            elif i <= 64:
                results['B'][i-32] = seats[i]
            elif i <= 95:
                results['C'][i-64] = seats[i]
            elif i <= 126:
                results['D'][i-95] = seats[i]
            elif i <= 157:
                results['E'][i-126] = seats[i]
            elif i <= 188:
                results['F'][i-157] = seats[i]
        
        for i in filled_seats:
            if i <= 32:
                results['A'][i] = 'A'
            elif i <= 64:
                results['B'][i-32] = 'A'
            elif i <= 95:
                results['C'][i-64] = 'A'
            elif i <= 126:
                results['D'][i-95] = 'A'
            elif i <= 157:
                results['E'][i-126] = 'A'
            elif i <= 188:
                results['F'][i-157] = 'A'

        #results.to_excel(r'C:\Users\Germa\OneDrive - Universidad de los andes\Libros U\2022-20 Flujo en Redes\resultados'+str(flightCounter)+'.xlsx')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-18
Time: 0.7498 Occupation %: 0.0 Blocked seats %: 79.7872
Time: 6.157 Occupation %: 1.06383 Blocked seats %: 79.7872
Time: 7.6233 Occupation %: 4.255319 Blocked seats %: 79.7872
Time: 1.208 Occupation %: 7.978723 Blocked seats %: 79.7872
Time: 1.269 Occupation %: 9.042553 Blocked seats %: 79.7872
Time: 1.1864 Occupation %: 9.574468 Blocked seats %: 79.7872
Time: 0.9719 Occupation %: 10.638298 Blocked seats %: 79.7872
Time: 0.9667 Occupation %: 11.170213 Blocked seats %: 79.7872
Time: 0.7551 Occupation %: 11.702128 Blocked seats %: 79.7872
Time: 0.8134 Occupation %: 12.234043 Blocked seats %: 79.7872
Time: 0.8037 Occupation %: 12.765957 Blocked seats %: 79.7872
Time: 1.0708 Occupation %: 13.297872 Blocked seats %: 79.7872
Time: 0.8987 Occupation %: 14.893617 Blocked seats %: 79.7872
Time: 0.9101 Occupation %: 16.489362 Blocked seats %: 79.7872
Time: 0.6545 Occupation %: 18.085106 Blocked seats %: 79